<img src="https://raw.githubusercontent.com/OGGM/oggm/master/docs/_static/logo.png" width="40%"  align="left">

# Specific mass-balance and ELA

Code for the OGGM blog post: http://oggm.org/2017/10/01/specmb-ela/

In [ ]:
# The commands below are just importing the necessary modules and functions
# Plot defaults
%matplotlib inline
import os
import matplotlib.pyplot as plt
# Scientific packages
import numpy as np
# Constants
from oggm.cfg import SEC_IN_YEAR, A
# OGGM models
from oggm.core.massbalance import LinearMassBalance
from oggm.core.flowline import FluxBasedModel, ParabolicBedFlowline
# This is to set a default parameter to a function. Just ignore it for now
from functools import partial
FlowlineModel = partial(FluxBasedModel, inplace=False)

pdir = os.path.expanduser('~')

## Set-up

### Glacier 

In [ ]:
# This is the bed rock, linearily decreasing from 3400m altitude to 1400m, in 200 steps
nx = 200
bed_h = np.linspace(3400, 1400, nx)
# At the begining, there is no glacier so our glacier surface is at the bed altitude
surface_h = bed_h
# Let's set the model grid spacing to 100m (needed later)
map_dx = 100
# For plot
grid = map_dx * np.arange(nx) * 1e-3
# The parabolic coeff
shape = surface_h * 0. + 5.e-3
# Define our bed
init_flowline = ParabolicBedFlowline(surface_h=surface_h, bed_h=bed_h, map_dx=map_dx, bed_shape=shape)

### Mass balance

We define a linear mass-balance model with a step function for the ELA:

In [ ]:
yr_end = 500
yrs = np.arange(yr_end+1)
ela_0 = 2900
ela_1 = ela_0 + 150
# Step function
sela = yrs * 0. + ela_0
sela[50:] = ela_1
class StepMassBalance(LinearMassBalance):
    """Constant mass-balance as a linear function of altitude."""
    def get_annual_mb(self, heights, year=None):
        self.ela_h = sela[int(year)]
        return super(StepMassBalance, self).get_annual_mb(heights, year=year)

And a linear mass-balance model with a linear change function for the ELA:

In [ ]:
# linear function
lela = yrs * 0. + ela_0
lela[50:-150] = np.linspace(ela_0, ela_1, len(lela[50:-150]))
lela[-150:] = ela_1
class LinChangeMassBalance(LinearMassBalance):
    """Constant mass-balance as a linear function of altitude."""
    def get_annual_mb(self, heights, year=None):
        self.ela_h = lela[int(year)]
        return super(LinChangeMassBalance, self).get_annual_mb(heights, year=year)

In [ ]:
ct_mb_model = LinearMassBalance(ela_0, grad=4)
step_mb_model = StepMassBalance(ela_0, grad=4)
lin_mb_model = LinChangeMassBalance(ela_0, grad=4)

### Equilibrium run

In [ ]:
emodel = FlowlineModel(init_flowline, mb_model=ct_mb_model, y0=0.)
emodel.run_until(1200)

See how the output looks like:

In [ ]:
# Plot the initial conditions first:
plt.figure();
plt.plot(grid, init_flowline.bed_h, color='k', label='Bedrock')
# The get the modelled flowline (model.fls[-1]) and plot it's new surface
plt.plot(grid, emodel.fls[-1].surface_h, label='Glacier in equilibrium')
plt.xlabel('Distance [km]')
plt.ylabel('Altitude [m]')
plt.xlim([0, 20]);
plt.legend(loc='best');
plt.tight_layout()
plt.savefig(os.path.join(pdir, 'setup.png'), dpi=150);

Let's print out a few infos about our glacier:

In [ ]:
print('Year:', emodel.yr)
print('Glacier length (m):', emodel.length_m)
print('Glacier area (km2):', emodel.area_km2)
print('Glacier volume (km3):', emodel.volume_km3)

## Step ELA 

In [ ]:
smodel = FlowlineModel(emodel.fls, mb_model=step_mb_model, y0=0.)
_, ds = smodel.run_until_and_store(yr_end)
# Keep only early vals
ds = ds.isel(time=ds.hydro_month==1)

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 5), sharex=True)
ds.ela_m.plot(ax=ax1, linewidth=3)
ax1.set_ylabel('ELA [m]')
ax1.set_xlabel('');
(ds.area_m2 * 1e-6).plot(ax=ax2, linewidth=3);
ax2.set_ylabel('Area [km$^2$]')
ax2.set_xlabel('');
smb = (ds.volume_m3.values[1:] - ds.volume_m3.values[:-1]) / ds.area_m2[:-1] * 900
smb.plot(ax=ax3, linewidth=3)
ax3.set_ylabel('SMB [kg m$^{-2}$]')
ax3.set_xlabel('Time [years]');
ax3.set_xlim([0, 500]);
plt.tight_layout()
plt.savefig(os.path.join(pdir, 'step-mb.png'), dpi=150);

## Linear ELA change 

In [ ]:
smodel = FlowlineModel(emodel.fls, mb_model=lin_mb_model, y0=0.)
_, ds = smodel.run_until_and_store(yr_end)
# Keep only early vals
ds = ds.isel(time=ds.hydro_month==1)

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 5), sharex=True)
ds.ela_m.plot(ax=ax1, linewidth=3, color='C3')
ax1.set_ylabel('ELA [m]')
ax1.set_xlabel('');
(ds.area_m2 * 1e-6).plot(ax=ax2, linewidth=3, color='C3');
ax2.set_ylabel('Area [km$^2$]')
ax2.set_xlabel('');
smb = (ds.volume_m3.values[1:] - ds.volume_m3.values[:-1]) / ds.area_m2[:-1] * 900
smb.plot(ax=ax3, linewidth=3, color='C3')
ax3.set_ylabel('SMB [kg m$^{-2}$]')
ax3.set_xlabel('Time [years]');
ax3.set_xlim([0, 500]);
plt.tight_layout()
plt.savefig(os.path.join(pdir, 'linear-mb.png'), dpi=150);